# Understanding the Out-of-Sample Error and the No Free Lunch Theorem

In this notebook, we will walk through the mathematics behind out-of-sample error and the No Free Lunch (NFL) Theorem. We will break down important equations and explore examples using Python to reinforce key concepts.


## Out-of-Sample Error

The out-of-sample error for a learning algorithm 
$$\mathcal{L}_a$$ 
is given by:

$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = \sum_{h} \sum_{x \in \mathcal{X} - X} P(x) \mathbb{I}(h(x) \neq f(x)) P(h | X, \mathcal{L}_a)
$$

Where:
- $
E_{\text{ote}}(\mathcal{L}_a | X, f) 
$ Is the out-of-sample error for a learning algorithm, given the training set and the ground truth.

- $
\mathcal{X} - X
$ Is the whole input space subtract the training set, which is therefore equivalent to all unseen data (i.e the test set).

- $
P(x)
$ Is the probability of observing (unseen/test) sample $x$.

- $
\mathbb{I}(h(x) \neq f(x))
$ Is the indicator function, which will be $0$ when our hypothesis (i.e. prediction) for unseen sample $x$ is equal to the ground truth, and $1$ otherwise.

- $
P(h | X, \mathcal{L}_a)
$ Is the probability of obtaining the hypothesis $h$ given the training data $X$ and the learning algorithm $\mathcal{L}_a$.

## Example in Python
Below is a very simple dummy calculation for the out-of-sample error using numpy:

In [1]:
import numpy as np

# Example data: input space (X) and hypothesis h
X = np.array([0, 1, 2, 3])
X_train = np.array([0, 1])  # Training set

# Probabilities for the data points
P_x = np.array([0.25, 0.25, 0.25, 0.25])

# True function f and hypothesis h
f = np.array([0, 1, 1, 0])  # Target function
h = np.array([0, 1, 0, 0])  # Hypothesis

# Indicator function: 1 if h(x) != f(x), else 0
indicator_function = (h != f).astype(int)

# Out-of-sample error: consider only points not in the training set
X_out_of_sample = np.setdiff1d(X, X_train)
out_of_sample_error = np.sum(P_x[X_out_of_sample] * indicator_function[X_out_of_sample])

print(f"Out-of-sample error: {out_of_sample_error}")


Out-of-sample error: 0.25


## Exploring NFL using Binary Classification problems
If we consider a binary classification problem, where our labels/outputs are either $1$ or $0$, we can view $f$ (our ground truth) as any function $x\rarr\{0, 1\}$.

This will have a function space of $\{0, 1\}^{|\mathcal{X}|}$, which expands to $\{(0, 0, 0, 0, ...), (1, 0, 0, 0, ...), ...\}$, where each tuple is a unique possible combination of outputs for the test data.

Using combanatorics, we know that the function space  $\{0, 1\}^{|\mathcal{X}|}$, must have size $2^{|\mathcal{X}|}$, as have $|\mathcal{X}|$ samples and $2$ possible choices for each sample, giving $2^{|\mathcal{X}|}$ possible combinations of label choices.

In [3]:
import numpy as np

# Number of samples |x|
n_samples = 4  # You can change this to any integer

# Generate all possible label combinations (0 or 1) for the n_samples
label_combinations = np.array(np.meshgrid(*[[0, 1]] * n_samples)).T.reshape(-1, n_samples)

# Print the possible combinations
print("All possible label combinations (function space):")
print(label_combinations)

# Check the total number of combinations, should be 2^n_samples
num_combinations = label_combinations.shape[0]
expected_combinations = 2 ** n_samples

print(f"\nNumber of combinations: {num_combinations}")
print(f"Expected number of combinations (2^{n_samples}): {expected_combinations}")

All possible label combinations (function space):
[[0 0 0 0]
 [0 1 0 0]
 [1 0 0 0]
 [1 1 0 0]
 [0 0 1 0]
 [0 1 1 0]
 [1 0 1 0]
 [1 1 1 0]
 [0 0 0 1]
 [0 1 0 1]
 [1 0 0 1]
 [1 1 0 1]
 [0 0 1 1]
 [0 1 1 1]
 [1 0 1 1]
 [1 1 1 1]]

Number of combinations: 16
Expected number of combinations (2^4): 16


# Uniform Distribution of Target Function
If we assume $f$ to be uniformly distributed (randomly assigning $0$ or $1$ with $0.5$ probability), then half of the possible functions $f \in \{0, 1\}^{|\mathcal{X}|}$ will produce the correct label for a given input.

This means $\frac{1}{2}2^{|\mathcal{X}|}$ samples will be incorrectly labeled (where $h(x) \neq f(x)$).

Meaning, we can simplify our out-of-sample error to:
$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = \frac{1}{2} 2^{|\mathcal{X}|} \sum_{h} \sum_{x \in \mathcal{X} - X} P(x) P(h | X, \mathcal{L}_a)
$$

By subtracting $1$ from $2^x$ we effectively half it, so we can simplify to:

$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = 2^{|\mathcal{X}| - 1} \sum_{h} \sum_{x \in \mathcal{X} - X} P(x) P(h | X, \mathcal{L}_a)
$$

We can re-arrange the sums:

$$
E_{\text{ote}}(\mathcal{L}_a | X, f) =  2^{|\mathcal{X}| - 1} \sum_{x \in \mathcal{X} - X} P(x) \sum_h P(h | X, \mathcal{L}_a) 
$$

Because $P(h | X, \mathcal{L}_a)$ is a probability distribution over h (i.e. the chance of obtaining each possible hypothesis in h), summing for all h with $\sum_h P(h | X, \mathcal{L}_a)$ must give a total probability of $1$.
So, we can simplify further into:

$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = 2^{|\mathcal{X}| - 1} \sum_{x \in \mathcal{X} - X} P(x) \cdot 1
$$

What we observe is our out-of-sample error for $\mathcal{L}_a$ is completely independent of $\mathcal{L}_a$.

This means for two different learners, $\mathcal{L}_a$ and $\mathcal{L}_b$:
$$
E_{\text{ote}}(\mathcal{L}_a | X, f) = E_{\text{ote}}(\mathcal{L}_b | X, f)
$$

Which would imply our choice of learner does not make a difference in error. No matter how 'smart' or 'dumb' a learner is, it has the same error as any other learner?